In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
from sklearn.metrics import r2_score

In [3]:
output_raw= pd.read_csv('data/배추_생산량.csv')
output_raw= output_raw.set_index('시도별')

In [17]:
df= pd.read_csv("data/최종_최종__월별기상관측.csv",encoding="cp949")
df= df.drop(labels="Unnamed: 0", axis=1)

In [ ]:
grouped= df.groupby(['행정구역', '일시']).mean()
grouped_fin= grouped
grouped_fin['면적']=0
grouped_fin['생산량']=0
grouped_fin['면적당 생산량']=0

In [ ]:
#면적 -> 생산량
city_list= ['강원도', '경기도', '인천광역시', '서울특별시', '경상북도', '충청북도', '충청남도', '대전광역시','전라남도', '대구광역시', '전라북도', '울산광역시', '경상남도', '광주광역시', '부산광역시', '제주도','세종특별자치시']
detail = ["", ".1", ".2", ".3", ".4", ".5", ".6", ".7", ".8", ".9", ".10", ".11"]
spr= ['-03', '-04', '-05']
sum= ['-06', '-07', '-08']
fal= ['-09', '-10', '-11']
win= ['-01', '-02', '-12']
season_list= [spr, sum, fal, win]

for i in range(len(city_list)):
    for l in range(2000, 2022):
        for k in range(len(detail)):
            try:
                val= str(l) + detail[k]
                for n in range(3):
                    dt= str(l) + spr[n]
                    grouped_fin.loc[(city_list[i], dt), '면적']= output_raw.loc[city_list[i], val]
            except:
                pass

In [18]:
output_raw

,2000,2000.1,2000.2,2000.3,2000.4,2000.5,2000.6,2000.7,2000.8,2001,...,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11
시도별,,,,,,,,,,,,,,,,,,,,,
품목별,일반봄배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),고랭지배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),노지가을배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),일반봄배추:면적 (ha),...,생산량 (톤),고랭지배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),노지가을배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),노지겨울배추:면적 (ha),10a당 생산량 (kg),생산량 (톤)
서울특별시,72,3695,2660,0,0,0,23,10152,2335,8,...,228,0,0,0,6,12793,771,0,0,0
부산광역시,403,4446,17917,0,0,0,324,10835,35105,452,...,259,0,0,0,36,11239,4017,0,0,0
대구광역시,177,4799,8494,0,0,0,139,10657,14813,232,...,1343,0,0,0,68,7248,4895,0,0,0
인천광역시,107,4316,4618,0,0,0,166,9907,16446,66,...,378,0,0,0,138,9664,13333,0,0,0
광주광역시,178,5090,9060,0,0,0,195,10437,20352,132,...,792,0,0,0,43,9772,4162,0,0,0
대전광역시,6,4696,282,0,0,0,99,9312,9219,26,...,154,0,0,0,59,8143,4821,0,0,0
울산광역시,61,4410,2690,0,0,0,91,10382,9448,49,...,503,0,0,0,85,7767,6577,0,0,0
세종특별자치시,-,-,-,-,-,-,-,-,-,-,...,487,0,0,0,50,9598,4839,0,0,0


In [13]:
#feature은 오직 면적, target은 생산량


In [9]:
output_raw

,2000,2000.1,2000.2,2000.3,2000.4,2000.5,2000.6,2000.7,2000.8,2001,...,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11
시도별,,,,,,,,,,,,,,,,,,,,,
품목별,일반봄배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),고랭지배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),노지가을배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),일반봄배추:면적 (ha),...,생산량 (톤),고랭지배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),노지가을배추:면적 (ha),10a당 생산량 (kg),생산량 (톤),노지겨울배추:면적 (ha),10a당 생산량 (kg),생산량 (톤)
서울특별시,72,3695,2660,0,0,0,23,10152,2335,8,...,228,0,0,0,6,12793,771,0,0,0
부산광역시,403,4446,17917,0,0,0,324,10835,35105,452,...,259,0,0,0,36,11239,4017,0,0,0
대구광역시,177,4799,8494,0,0,0,139,10657,14813,232,...,1343,0,0,0,68,7248,4895,0,0,0
인천광역시,107,4316,4618,0,0,0,166,9907,16446,66,...,378,0,0,0,138,9664,13333,0,0,0
광주광역시,178,5090,9060,0,0,0,195,10437,20352,132,...,792,0,0,0,43,9772,4162,0,0,0
대전광역시,6,4696,282,0,0,0,99,9312,9219,26,...,154,0,0,0,59,8143,4821,0,0,0
울산광역시,61,4410,2690,0,0,0,91,10382,9448,49,...,503,0,0,0,85,7767,6577,0,0,0
세종특별자치시,-,-,-,-,-,-,-,-,-,-,...,487,0,0,0,50,9598,4839,0,0,0
